In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw8.ipynb")

In [2]:
import numpy as np
import torch
import torch.nn as nn
import gymnasium as gym

# FILL IN YOUR NAME AND THE NAME OF YOUR PEER (IF ANY) BELOW

**Name**: Maede Boroji

**Peer**: 

## Collaboration policy
Students are responsible for writing their own quizzes, assignments, and exams. For homework assignments, students are welcome (and encouraged) to discuss problems with one peer, **but each student must write their own assignment wrtieup and code individually**. The peer must be listed at the top of the writeup for each assignment. *Note: I will treat AI assistants as peers. That is, students are welcome to discuss problems with an AI assistant, but it is considered cheating to directly obtain an answer by querying the assistant. Please credit any AI assistant that you use.*

# Homework 8 -- Policy gradient learning (100 pts)

**Due:** Tuesday, April 22th, 2025 at 11:59 pm

*HW credit: This homework is heavily based on UC Berkeley's CS294-112 HW2 from Fall '18.*

This homework builds on the material in the slides and Sutton & Barto Chapter Chapter 13.

We will use Jupyter/Colab notebooks throughout the semester for writing code and generating assignment outputs.

**This homework will be unlike prior homeworks. It will be _entirely_ implementation-based. Some questions will be assessed by running your code, while others will require you to upload trained neural nets, which the grader will evaluate.**

## 1) Implementing REINFORCE

Recall that the reinforcement learning objective is to learn a $W*$ that maximizes the objective function:
$$J(W) = \mathbb{E}\left[\sum_{t}\gamma^t R(S_t, A_t)\right]\enspace.$$

The policy gradient approach is to directly take the gradient of this objective:
$$ \nabla J(\theta) \propto \mathbb{E}\left[G_t \frac{\nabla \pi_W(A_t \mid S_t)}{\pi_W(A_t \mid S_t)}\right]\enspace,$$

where $G_t=\sum_{k=t}^T \gamma^{k-t}R(S_t, A_t)$.

In practice, the expectation can be approximated from a batch of $N$ sampled trajectories:
$$
    \nabla J(W) \approx \frac{1}{N} \sum_{i=1}^{N} \sum_{t=1}^{T} G_{t,i} \frac{\nabla\pi_W(A_{t,i} \mid S_{t,i})}{\pi_W(A_{t,i} \mid S_{t,i})}\enspace.
$$

Here we see that the policy $\pi_W$ is a probability distribution over the action space, conditioned on the state. In the agent-environment loop, the agent sampled an action $A_t$ from $\pi_W(\cdot \mid S_t)$, and the environment responds with a reward $R(S_t, A_t)$. 

We saw in the lecture that subtracting a baseline that (possibly) depends on the state $S_t$ but not on the action $A_t$ does not change the expectation of the gradient, so we can alternatively compute:
$$
    \nabla J(W) \approx \frac{1}{N} \sum_{i=1}^{N} \sum_{t=1}^{T} (G_{t,i}-b(S_{t,i}))\frac{\nabla\pi_W(A_{t,i} \mid S_{t,i})}{\pi_W(A_{t,i} \mid S_{t,i})}\enspace.
$$

In this assignment, we will implement a value function $V_\theta^\pi$ that acts as a state-dependent baseline. The value function is trained to approximate the sum of future rewards starting from a particular state:
$$
    V_\theta^\pi(S_t) \approx \sum_{k=t}^T \mathbb{E}[\gamma ^{k-t}R(S_k, A_k)]
$$


In this problem, we will implement the REINFORCE algorithm both with and without a baseline, as well as try a few other techniques to reduce the variance of our gradient estimates (and therefore improve the quality of policy gradient training). 

### 1.1) Gaussian policy

Your first task is to implement a Gaussian policy class. The class constructor will take as input the following arguments:
- `obs_dim`: the number of features in the state
- `action_dim`: the dimensionality of the action space
- `hid_size`: the number of neurons in the hidden layers
- `num_layers`: the number of hidden layers in your network. (*Note: in NN-speak, the "input" layer is the actual features, the "output" layer is the final set of nodes, and the "hidden" layres are all the ones in between.*)

The policy will use a neural net based on the arguments described above to compute the mean $\mu(S_t)$ of a Gaussian, and will use a trainable parameter $\log\sigma$ to represent the log of the standard deviation of each dimension of the action space. 

You can use your `FCNN` function from HW 7 to create your $\mu$ network **but this time around we will use `nn.Tanh` as the activation function**. You should use `nn.Parameter` to define your trainable (but not state-dependent) log-standard-dev. 

_Points:_ 10

In [3]:

class GaussianPolicy_11(nn.Module):
    def __init__(self, obs_dim, action_dim, hid_size, num_layers):
        super().__init__()
        Layers = nn.ModuleList()
        current_dim = obs_dim
        for _ in range(num_layers):
            Layers.append(nn.Linear(current_dim, hid_size))
            Layers.append(nn.Tanh())
            current_dim = hid_size
        Layers.append(nn.Linear(current_dim, action_dim))
        self.mu = nn.Sequential(*Layers)
        # Initialize the log std to all zeros
        self.log_std = nn.Parameter(torch.zeros(action_dim))

    def forward(self, obs):
        ''' 
        obs: Tensor of shape (batch_size, obs_dim)
        return: mu: Tensor of shape (batch_size, action_dim)
                std: Tensor of shape (action_dim,)
        '''
        mu = self.mu(obs)
        std = torch.exp(self.log_std)
        return mu, std

    def sample(self, obs):
        '''
        obs: Tensor of shape (batch_size, obs_dim)
        return: action: Tensor of shape (batch_size, action_dim)

        Hint: Look into torch.distributions.Normal
        '''
        mu, std = self(obs)
        dist = torch.distributions.Normal(mu, std)
        action = dist.sample()
        return action

    def log_prob(self, obs, action):
        '''
        obs: Tensor of shape (batch_size, obs_dim)
        action: Tensor of shape (batch_size, action_dim)
        return: log_prob: Tensor of shape (batch_size,)

        Hint 1: You may use torch.distributions.Normal.log_prob
        Hint 2: Think about how the joint probability of independent variables is 
        computed and then how you may do this in log space.
        '''
        mu, std = self(obs)
        dist = torch.distributions.Normal(mu, std)
        log_prob = dist.log_prob(action).sum(dim=-1)
        return log_prob


### 1.2) Sample a trajectory

You will now implement a function that samples a collection of trajectories. This function should be nearly identical to the `run_policy` function from HW7, with the difference that it will instead return a dictionary containing the data from the trajectory. 

I have provided below a `sample_trajectories` function that loops over calls to your `sample_trajectory` function to collect all the data and put it in the format we will need in subsequent functions.

_Points:_ 5

In [6]:
def sample_trajectory_12(env, policy, seed=None):
    obs, _ = env.reset(seed=seed)
    observations, actions, rewards = [], [], []
    steps = 0
    # ***Your code here:***
    # Note: you should discard the final observation (the one that is accompanied by a termination or truncation)
    
    while True:
        action = policy.sample(torch.tensor(obs, dtype=torch.float32).unsqueeze(0)).numpy().flatten()
        next_obs, reward, terminated, truncated, _ = env.step(action)
        observations.append(obs)
        actions.append(action)
        rewards.append(reward)
        obs = next_obs
        steps += 1
        if terminated or truncated:
            break
    

    '''
    observations: (steps, obs_dim)
    actions: (steps, action_dim)
    rewards: (steps,) 
    length: int = steps
    '''
    observations = np.array(observations, dtype=np.float32)
    actions = np.array(actions, dtype=np.float32)
    rewards = np.array(rewards, dtype=np.float32)
    path = {'observations': torch.from_numpy(observations),
            'actions': torch.from_numpy(actions),
            'rewards': torch.from_numpy(rewards),
            'length': steps}
    return path

# DO NOT MODIFY THE CODE BELOW
def sample_trajectories_12(env, policy, min_batch_size, seed=None):
    timesteps = 0
    paths = []
    itr = 0
    while timesteps < min_batch_size:
        with torch.no_grad():   # accelerate computation by turning off unnecessary gradients
            path = sample_trajectory_12(env, policy, seed=seed+itr*1000)
        itr += 1
        paths.append(path)
        timesteps += path['length']
    return paths, timesteps

### 1.3) Computing the cumulative rewards

You will now write code to compute $G_t=\sum_{k=t}^T \gamma^{k-t}R(S_t, A_t)$. 

_Points:_ 10

In [7]:
def sum_of_rewards_13(rewards, gamma):
    ''' 
    rewards: list of torch tensors, each of which is the rewards for a single trajectory
    gamma: float, the discount factor
    return: torch tensor of shape (num_trajectories * num_steps, ), the reward-to-go for each time step, flattened
    '''
    all_rewards = []
    for trajectory in rewards:
        path_length = len(trajectory)
        rewards = torch.zeros(path_length)
        for t in reversed(range(path_length)):
            rewards[t] = trajectory[t]
            if t < path_length - 1:
                rewards[t] += gamma * rewards[t + 1]
        all_rewards.append(rewards)
        
    all_rewards = torch.cat(all_rewards)
    return all_rewards

### 1.4) Loss function

We derived the policy gradient to be:
$$
    \nabla J(W) \approx \frac{1}{N\cdot T} \sum_{i=1}^{N} \sum_{t=1}^{T} G_{t,i} \frac{\nabla\pi_W(A_{t,i} \mid S_{t,i})}{\pi_W(A_{t,i} \mid S_{t,i})}\enspace.
$$

In order to leverage the autodiff capabilities of PyTorch, we need to write a *loss* function to do gradient **descent**. How should the gradient of the loss function relate to the gradient defined above? 

Write a loss function whose gradient is as desired for gradient descent.

(Note: we made a minor change in our derivation to include a factor of $\frac{1}{T}$. While this theoratically doesn't change anything---we can always scale the learning rate appropriately independently of what multiplicative factor we use---this choice makes implementation easier. Make sure that your loss function incorporates the appropriate scale in order for the autograder to work correctly, and for your learning rate to match the ones suggested in the assignment.)

_Points:_ 10

In [ ]:
def pg_loss_fn_14(policy, observations, actions, returns):
    ''' 
    policy: GaussianPolicy_11 object
    observations: Tensor of shape (num_trajectories * num_steps, obs_dim)
    actions: Tensor of shape (num_trajectories * num_steps, action_dim)
    returns: Tensor of shape (num_trajectories * num_steps, )
    return: loss: Tensor of shape (1, ), a loss whose gradient can be used for gradient *descent*
    '''
    log_prob = policy.log_prob(observations, actions)
    loss = -torch.mean(log_prob * returns)  
    return loss


### 1.5) Putting it all together: PG training

You will now complete the code below that implements PG training by putting together all the functions you have written so far. 

The function `train_PG_15` will implement the training loop, which internally calls the function `update_parameters_15`. The latter function takes one gradient descent step using the optimizer.

In this question, you will implement one additional trick in `train_PG_15`: normalizing the returns. In particular, if the argument `normalize_returns=True`, your code should normalize the returns to have mean zero and standard deviation one. 

The parts of the code that you should modify are annotated with `*** 1.5 -- YOUR CODE HERE ***`.

*Note: parts of this code are annotated with `*** 2.3 -- YOUR CODE HERE ***`. These parts of the code will not be tested in this question, but in later questions. You may leave those parts of the code unchanged until you reach the relevant problem.*

_Points:_ 10

In [ ]:
def update_parameters_15(policy, observations, actions, returns, optimizer):
    # *** 1.5 YOUR CODE HERE ***
    loss = pg_loss_fn_14(policy, observations, actions, returns)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.detach().item()     # You may print this in your training function for debugging

def train_PG_15(env_name, 
                hid_size=64, 
                num_layers=2, 
                use_baseline=False, 
                num_iterations=100, 
                batch_size=1000,
                gamma=0.99,
                normalize_returns=False,
                learning_rate=1e-3,
                seed=0):
    # Make the gym environment
    env = gym.make(env_name)

    # Set the random seed
    torch.manual_seed(seed)
    np.random.seed(seed)

    # *** 1.5 YOUR CODE HERE ***
    policy = GaussianPolicy_11(obs_dim=env.observation_space.shape[0], action_dim=env.action_space.shape[0], hid_size=hid_size, num_layers=num_layers)
    optimizer = torch.optim.Adam(policy.parameters(), lr=learning_rate)

    if use_baseline:
        # *** 2.3 YOUR CODE HERE ***
        nn_baseline = ...
        optimizer_baseline = ...
    
    total_timesteps = 0
    for iter in range(num_iterations):
        print(f"********** Iteration {iter} **********")

        # Sample trajectories
        # *** 1.5 YOUR CODE HERE ***
        paths, timesteps = sample_trajectories_12(env, policy, batch_size, seed=seed+iter*1000)
        total_timesteps += timesteps

        # Build tensors
        observations = torch.cat([path['observations'] for path in paths], dim=0)
        actions = torch.cat([path['actions'] for path in paths], dim=0)
        rewards = [path['rewards'] for path in paths]

        # Print the average undiscounted return
        undiscounted_return = torch.cat(rewards).sum() / len(paths)
        print(f"\tAverage return: {undiscounted_return:.3f}")

        # Compute the reward-to-go
        # *** 1.5 YOUR CODE HERE ***
        rewards_to_go = sum_of_rewards_13(rewards, gamma)

        if use_baseline:
            # *** 2.3 YOUR CODE HERE ***
            baseline = ...
            returns = ...
        else:
            returns = rewards_to_go
        if normalize_returns:
            # *** 1.5 YOUR CODE HERE ***
            # Normalize the returns
            # Hint: Add a small value of 1e-5 to the denominator to avoid division by zero
            returns = (returns - returns.mean()) / (returns.std() + 1e-5)

        # Update the policy
        # *** 1.5 YOUR CODE HERE ***
        loss = update_parameters_15(policy, observations, actions, returns, optimizer)
        print(f"\tPolicy loss: {loss:.3f}")

        if use_baseline:
            # *** 2.3 YOUR CODE HERE ***
            # Update the baseline
            ...

    return policy

### 1.6) Experiments

Run experiments using `env_name = "InvertedPendulum-v5"`. Use the following arguments to your `train_PG` function:
- `hid_size`: 64
- `num_layers`: 2
- `num_iterations`: 100
- `batch_size`: ?
- `gamma`: 0.99
- `normalize_returns`: ?
- `learning_rate`: ?
- `seed`: 0

Find values for the `batch_size`, `normalize_returns`, and `learning_rate` such that the agent achieves the optimal sum of rewards for this environment (1000) in less than 100 iterations. For your choice of hyperparameters, your code should run in just a few seconds --- this limits the `batch_size` that you can choose, or the autograder will time out. Some suggested values to try are included below:
- `batch_size in [10, 100, 1000, 3000]`
- `learning_rate in [1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2]`
- `normalize_returns in [True, False]`

Report your chosen hyperparameters below.

The code cell below includes code to visualize your learned policy, which you can use throughout this assignment to observe the behaviors that your learned policy achieves.

_Points:_ 15

In [12]:
def visualize_policy(env_name, policy, num_episodes=1):
    env = gym.make(env_name, render_mode="human")
    for episode in range(num_episodes):
        obs, _ = env.reset()
        done = False
        while not done:
            action = policy.sample(torch.from_numpy(obs.astype(np.float32)))
            action = action.detach().numpy()
            obs, reward, terminated, truncated, _ = env.step(action)
            env.render()
            done = terminated or truncated
    env.close()

In [13]:
batch_size_16 = 1000
normalize_returns_16 = True
learning_rate_16 = 0.02

In [ ]:
#train and visualize the policy
# policy_16 = train_PG_15(env_name="InvertedPendulum-v5", 
#                         hid_size=64, 
#                         num_layers=2, 
#                         use_baseline=False, 
#                         num_iterations=100, 
#                         batch_size=batch_size_16,
#                         gamma=0.99,
#                         normalize_returns=normalize_returns_16,
#                         learning_rate=learning_rate_16,
#                         seed=0)

********** Iteration 0 **********
	Average return: 7.556
	Policy loss: -0.079
********** Iteration 1 **********
	Average return: 24.000
	Policy loss: -0.036
********** Iteration 2 **********
	Average return: 15.111
	Policy loss: -0.001
********** Iteration 3 **********
	Average return: 26.324
	Policy loss: -0.023
********** Iteration 4 **********
	Average return: 32.161
	Policy loss: -0.005
********** Iteration 5 **********
	Average return: 29.303
	Policy loss: -0.018
********** Iteration 6 **********
	Average return: 31.258
	Policy loss: -0.003
********** Iteration 7 **********
	Average return: 50.600
	Policy loss: 0.013
********** Iteration 8 **********
	Average return: 63.812
	Policy loss: -0.000
********** Iteration 9 **********
	Average return: 90.364
	Policy loss: -0.030
********** Iteration 10 **********
	Average return: 72.214
	Policy loss: -0.028
********** Iteration 11 **********
	Average return: 76.071
	Policy loss: -0.006
********** Iteration 12 **********
	Average return: 

In [23]:
# save trained policy in pt file
torch.save(policy_16.state_dict(), "trained_Pendulum_policy.pt")

In [ ]:
# visualize_policy("InvertedPendulum-v5", policy_16, num_episodes=1)

c:\Users\boroj\miniforge3\envs\py310\lib\site-packages\glfw\__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


## 2) REINFORCE with baseline

We will now implement a value function as a state-dependent neural network baseline. This will allow us to reduce the variance of the gradient computation and hopefully learn to solve significantly harder problems. 

You will implement the vanilla version, which simply uses $\mathcal{L}(\theta) = \frac{1}{NT}\sum_{i=1}^N\sum_{t=1}^T(G_t - V^\pi_\theta(s_t))^2$, where $G_t$ is the reward-to-go measured directly from the trajectories.

For this, it will be useful to apply the following normalization trick:
- During training of $V^\pi_\theta$, instead of using the reward-to-go $G_t$ as the target labels, you will first normalize the reward-to-go to have mean zero and standard deviation 1. As we have seen in HW7, this is generally a good idea for NN training. But it is particularly helpful in this case where the scale of the reward-to-go may change over time --- we certainly hope that reward goes up as the agent learns!
- When applying the baseline to compute the policy gradient, you will compute the baseline values and scale them so that their mean and standard deviation match those of the reward-to-go in the current batch. 

### 2.1) Computing the baseline

Your first task is to compute, given a NN baseline $V^\pi_\theta$, the baseline value $V_\theta^\pi(s_t)$. Make sure that the baseline value is appropriately scaled to have the same mean and standard deviation as the reward-to-go.

As usual, you should take care not to divide by zero by adding 1e-5 to any denominator that could go to zero.

_Points:_ 5

In [ ]:
def compute_baseline_21(nn_baseline, observations, rewards_to_go):
    ''' 
    nn_baseline: FCNN_11 object
    observations: Tensor of shape (num_trajectories * num_steps, obs_dim)
    rewards_to_go: Tensor of shape (num_trajectories * num_steps,)
    return: Tensor of shape (num_trajectories * num_steps,) the baseline values computed from nn_baseline, with the same mean and standard dev as returns
    '''
    with torch.no_grad():   # We don't backgprop through the baseline when computing PGs
        ...

### 2.2) Computing the baseline loss

You will now compute the loss function to train the baseline on. This will be the mean squared error of the predictions of the baseline compared to the reward-to-go. To simplify the training, you will normalize the reward-to-go targets to have zero mean and standard deviation one.

_Points:_ 5

In [ ]:
def baseline_loss_22(nn_baseline, observations, rewards_to_go):
    ''' 
    nn_baseline: FCNN_11 object
    observations: Tensor of shape (num_trajectories * num_steps, obs_dim)
    rewards_to_go: Tensor of shape (num_trajectories * num_steps,)
    return: singleton Tensor of shape ([]), the mean square error

    Hint: Think about the shapes of the output of nn_baseline and the rewards_to_go, and 
    how this would work if you use broadcasting vs manual reshaping.
    '''
    ...

### 2.3) Putting it all together: PG training with baseline

You will now complete the code below and in the `train_PG_15` function (from Problem 1.5) to incorprate the baseline training into the PG loop. Complete any parts annotated with `*** 2.3 YOUR CODE HERE ***`. 

_Points:_ 10

In [ ]:
def update_baseline_parameters_23(nn_baseline, observations, rewards_to_go, optimizer):
    # *** 2.3 YOUR CODE HERE ***
    loss = ...
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.detach().item()     # You may print this in your training function for debugging

### 2.4) Experiments with more complex tasks

**Note:** The following tasks will take quite a bit of time to train. Please start early!

Run experiments using `env_name = "HalfCheetah-v5"`. Use the following arguments to your `train_PG` function:
- `hid_size`: 32
- `num_layers`: 2
- `use_baseline`: ?
- `num_iterations`: 100
- `batch_size`: ?
- `gamma`: 0.95
- `normalize_returns`: True
- `learning_rate`: ?
- `seed`: 0

Find values for the `use_baseline`, `batch_size`, and `learning_rate` such that the agent achieves the highest sum of rewards that you can for this environment in less than 100 iterations. Some suggested values to try are included below:
- `batch_size in [10000, 30000, 50000]`
- `learning_rate in [1e-3, 3e-3, 1e-2, 3e-2, 1e-3]`
- `use_baseline in [True, False]`

You should be able to obtain sum of rewards above 1800 with an appropriate choice of hyperparameters.

Unlike Problem 1.6, this time you should train your policy in a separate notebook and save it as `trained_cheetah_policy.pt`. Because PG training is quite noisy, it is unlikely that the policy at iteration 100 is the best performing one, so you may want to add the following lines to your `train_PG` function in your other notebook:

```
...
total_timesteps = 0
highest_returns = -np.inf
for iter in range(num_iterations):
    ... 
    undiscounted_return = rewards.sum() / len(paths)
    if undiscounted_return > highest_returns:
        torch.save(policy.state_dict(), 'trained_cheetah_policy.pt')
        highest_returns = undiscounted_return
    ...
```

_Points:_ 20

In [ ]:
# Nothing to do in this code cell. Please run your training and saving code in a separate notebook.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Fill out the answers to all questions. Submit a zip file containing hw8.ipynb with your answers and the `trained_cheetah_policy.pt` file you saved to the HW8 assignment on Gradescope. You are free to resubmit as many times as you wish.

In [24]:
# Save your notebook first, then run this cell to export your submission.
#grader.export(pdf=False, run_tests=True, files=['trained_cheetah_policy.pt'])
grader.export(pdf=False, run_tests=True, files=['trained_Pendulum_policy.pt'])
#grader.export(pdf=False, run_tests=True)

Running your submission against local test cases...


Your submission received the following results when run against available test cases:


